# Data

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import geopandas as gpd
import os
from functools import reduce
from zonal_stats import *

In [2]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [3]:
# Move working directory to Data folder
os.chdir('/Tesis/Datos') 
os.getcwd()
pkls = '/Tesis/Codes/pkls/'

## Metropolitan areas

The metropolitan areas used for this work are those defined by the CONAPO in 2015.
This file was downloaded from:

https://datos.gob.mx/busca/dataset/distribucion-territorial

In [4]:
# Read 2015 and 2010 csv files and extract Metropolis
ZM_2015 = pd.read_csv("Zonas metropolitanas\ZM_2015.csv", encoding='latin-1', usecols=list(range(7)))              
ZM_2010 = pd.read_csv("Zonas metropolitanas\Base_delimitacionZM_00-10.csv", usecols=[2,21])

# Select 2010 values and set "2015" to False if new Metropoli
ZM_2010 = ZM_2010.loc[ZM_2010['AÑO']==2010]
ZM_2015['2015'] = ZM_2015['CVE_MUN'].isin(ZM_2010['CVE_MUN'])
del ZM_2010

ZM_2015.to_pickle(pkls + 'ZM_2015.pkl')

ZM_2015.head()

,CVE_ZM,NOM_ZM,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN,POB_2015,2015
0,1.01,Aguascalientes,1,Aguascalientes,1001,Aguascalientes,877190,True
1,1.01,Aguascalientes,1,Aguascalientes,1005,Jesús María,120405,True
2,1.01,Aguascalientes,1,Aguascalientes,1011,San Francisco de los Romo,46454,True
3,2.01,Ensenada,2,Baja California,2001,Ensenada,486639,False
4,2.02,Mexicali,2,Baja California,2002,Mexicali,988417,True


In [9]:
ZM_2015.groupby('CVE_ZM').filter(lambda x: (x['2015'] == False).all())

,CVE_ZM,NOM_ZM,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN,POB_2015,2015
3,2.01,Ensenada,2,Baja California,2001,Ensenada,486639,False
8,3.01,La Paz,3,Baja California Sur,3003,La Paz,272711,False
9,4.01,Campeche,4,Campeche,4002,Campeche,283025,False
31,7.01,Tapachula,7,Chiapas,7089,Tapachula,348156,False
40,8.02,Delicias,8,Chihuahua,8021,Delicias,148045,False
41,8.02,Delicias,8,Chihuahua,8045,Meoqui,44752,False
42,8.03,Hidalgo del Parral,8,Chihuahua,8032,Hidalgo del Parral,109510,False
43,8.03,Hidalgo del Parral,8,Chihuahua,8059,San Francisco del Oro,5086,False
121,10.01,Durango,10,Durango,10005,Durango,654876,False
126,11.02,Guanajuato,11,Guanajuato,11015,Guanajuato,184239,False


## Temperature

This section takes the average seasonal temperature for every municipality of the Metropolitan zones.
To do so, a raster with the average temperature from 1902 to 2011 is used. Using zonal stats, the mean per municipality is taken for every month and aggregated by season.

Raster: http://atlasclimatico.unam.mx/atlas/kml/

In [21]:
# Create a shapefiles for municipalities if there isn't one
fname = "shapefiles/MUN/MUN_ZM.shp"
if not os.path.exists(fname):
        # Get individual municipalities shape files in directory
    mun_geo_path = 'Marco Geoestadistico/conjunto de datos'
    mun_files = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(mun_geo_path):
        for file in f:
            if 'mun.shp' in file:
                mun_files.append(os.path.join(r, file))
    # Put all files in a list to concatenate it
    mun_list = []
    for file in mun_files:
        mun_x = gpd.read_file(file)
        mun_list.append(mun_x)
        
    # Concatenate    
    mun = pd.concat(mun_list, sort=False)
    mun.reset_index(inplace=True, drop=True)
    
    # Fix Municipalities code
    mun['CVE_MUN'] = (mun['CVE_ENT'] + mun['CVE_MUN']).astype(np.int64)
    # Take municipalities within metropolitan zones
    mun_ZM = mun.loc[mun['CVE_MUN'].isin(ZM_2015['CVE_MUN'])].copy()

    # Create shapefile
    mun_ZM.to_file(fname)
    print(fname + 'created')

# Create a df with the mean temperature in the municipality
vector_path = fname
#vector_path = "shapefiles/MUN/MUN.shp"
temp_list = []
for i in range(1,13):
    raster_path = 'Temperatura/TemperaturaMedia_mensual_raster/tmedia_'+ str(i) + '/tmedia_' + str(i) + '/Geotiff/tmed_' + str(i) + '.tif'
    temp_x = (pd.DataFrame.from_dict(zonal_stats(vector_path, raster_path, 'population','CVE_MUN', nodata_value=-3.4028234663852886e+38)))
    temp_x.rename(columns={'mean':str(i)}, inplace=True)
    temp_list.append(temp_x)
    
# Merge months in one df
temp_merged = reduce(lambda  left,right: pd.merge(left,right,on=['CVE_MUN']), temp_list)
temp_merged['CVE_MUN'] = temp_merged['CVE_MUN'].astype(np.int64)
temp_merged.to_pickle(pkls + 'temp_months_mun.pkl')

# Reduce the months to seasons
temp_season = pd.DataFrame()
temp_season['CVE_MUN'] = temp_merged['CVE_MUN'].astype(np.int64)
temp_season['Winter'] = temp_merged.loc[:,['1','2','12']].sum(axis = 1)/3 
temp_season['Spring'] = temp_merged.loc[:,'3':'5'].sum(axis = 1)/3 
temp_season['Summer'] = temp_merged.loc[:,'6':'8'].sum(axis = 1)/3 
temp_season['Fall'] = temp_merged.loc[:,'9':'11'].sum(axis = 1)/3
temp_season['max'] = temp_merged.loc[:,'4':'9'].max(axis=1)

temp_season.to_pickle(pkls + 'temp_season_mun.pkl')

# Merge Temp with ZM dataframe
ZM_2015 = ZM_2015.merge(temp_season, how='left', on=['CVE_MUN'])
ZM_2015.reset_index(inplace=True, drop=True)

del temp_season

ZM_2015.head()

,CVE_ZM,NOM_ZM,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN,POB_2015,2015,Winter,Spring,Summer,Fall,max
0,1.01,Aguascalientes,1,Aguascalientes,1001,Aguascalientes,877190,True,13.242611,18.714286,20.383949,17.192980,21.284910
1,1.01,Aguascalientes,1,Aguascalientes,1005,Jesús María,120405,True,12.991325,18.370768,19.818192,16.760830,20.828707
2,1.01,Aguascalientes,1,Aguascalientes,1011,San Francisco de los Romo,46454,True,12.995048,18.727809,20.463810,17.045714,21.378287
3,2.01,Ensenada,2,Baja California,2001,Ensenada,486639,False,12.520194,15.755021,23.148775,19.307285,24.536871
4,2.02,Mexicali,2,Baja California,2002,Mexicali,988417,True,13.152247,18.867188,28.126219,21.820882,29.293267


### Tempeture for tariff map CFE
This cell creates a temperature shapefile according to CFE's tariffs

In [22]:
# Merge municipalities shapefile with min and max temperatures
temp_merged['CVE_MUN'] = temp_merged['CVE_MUN'].astype(np.int64)
mun = gpd.read_file("shapefiles/MUN/MUN.shp")
mun_tarifa = mun.merge(temp_merged[['CVE_MUN','4','5','6','7','8','9']], on=['CVE_MUN'])
mun_tarifa['min'] = temp_merged.loc[:,'4':'9'].min(axis=1)
mun_tarifa['max'] = temp_merged.loc[:,'4':'9'].max(axis=1)
mun_tarifa.to_file("shapefiles/MUN/Temperatura/MUN_tarifa.shp") 

del temp_merged

## Censo de Población y Vivienda 2010

The CPV 2010 is the last complete census available. The files are available for each state. The highest level of disaggregation corresponds to "manzanas" (blocks). These files are read and added to the Metropolis dataframe.

These files were downloaded from:
https://www.inegi.org.mx/programas/ccpv/2010/default.html#Datos_abiertos

In [23]:
# Columns to take from Censo de Población y Vivienda
cpyv_col = [0,2] + list(range(4,12)) + [29,32,35,38,41,44,26,48,54] + [108,129] + \
            [132,135,138,141] + list(range(157,165)) + [171,172] + list(range(189,198))

# Read Censo de Población y Vivienda files
CPV_path = 'Censo Poblacion y Vivienda 2010/AGEB/conjunto_de_datos'

# Locate filenames
CPV_files = []
for r, d, f in os.walk(CPV_path):
    for file in f:
        CPV_files.append(os.path.join(r, file))

# Put all csv files in a list to concatenate it
CPVlist = []
for file in CPV_files:
    CPV_ent = pd.read_csv(file, usecols=cpyv_col, dtype={'ageb': str}, na_values=['*','N/D'])
    CPVlist.append(CPV_ent)
CPV = pd.concat(CPVlist)

# Create a new CVE_MUN index
CPV['mun'] = CPV['entidad']*1000 + CPV['mun']

# Set names for the merging
CPV.rename(columns={'entidad':'CVE_ENT', 'loc':'CVE_LOC', 'mun':'CVE_MUN'}, inplace=True)

# Create missing age range
CPV['p_25a59'] = CPV['p_18ymas']-CPV['p_18a24']-CPV['p_60ymas']

# Organize columns
cols = CPV.columns.tolist()
cols = cols[:cols.index('p_60ymas')] + ['p_25a59'] + cols[cols.index('p_60ymas'):-1]
CPV = CPV[cols]

# Merge CPV with ZM dataframe
ZM_CPV = ZM_2015.merge(CPV, how='left', on=['CVE_MUN','CVE_ENT'])
ZM_CPV.reset_index(inplace=True, drop=True)
del CPV

## Grado de marginación urbana

The stage of urban marginalization is available at AGEB level, the stages are:
- Very low
- Low
- Medium
- High
- Very high

The data is added to our previous Metropolis dataframe

These files were downloaded from:
https://datos.gob.mx/busca/dataset/indice-de-marginacion-carencias-poblacionales-por-localidad-municipio-y-entidad

In [24]:
# Columns to take from Grado de marginación urbana
GMU_cols = [6,30,31,32] 

# File path
GMU_file = 'Marginacion\Base_marginacion_AGEB_00-10.csv'

# Read GMU file and take year 2010
GMU = pd.read_csv(GMU_file, usecols=GMU_cols, encoding='latin-1')
GMU = GMU.loc[GMU['AÑO']==2010].reset_index(drop=True)

# Explode CVE_AGEB into columns to create matching index to ZM_2015
GMU['CVE_ENT'] = GMU['CVE_AGEB'].str[:-11].astype(np.int64)
GMU['CVE_MUN'] = GMU['CVE_AGEB'].str[:-8].astype(np.int64)
GMU['CVE_LOC'] = GMU['CVE_AGEB'].str[-8:-4].astype(np.int64)
GMU['ageb'] = GMU['CVE_AGEB'].str[-4:]
del GMU['CVE_AGEB'], GMU['AÑO']

# Fix negative values in IMU
GMU.loc[(GMU['GMU']=='Muy bajo') | (GMU['GMU']=='Bajo') | (GMU['GMU']=='Medio'),'IMU'] = -GMU['IMU']

GMU.to_pickle(pkls + 'GMU.pkl')

# Join ZM_2015 with GMU
ZM_GMU = ZM_CPV.merge(GMU, how='left', on=['CVE_ENT', 'CVE_MUN', 'CVE_LOC', 'ageb'])
del GMU

ZM_GMU.to_pickle(pkls + 'ZM_GMU.pkl')

### Clean and prepare data for regression
This cell clean the data and fill nans in order to use the CPV, temperature, and IMU variables as regressors in the regression model

In [25]:
# Take columns of interest
cols = ['CVE_ZM','CVE_ENT','CVE_MUN','CVE_LOC','ageb','mza','Summer','pobtot',
        'tothog','prom_ocup','IMU','graproes','pocupada'] + ZM_GMU.columns[18:31].tolist() 

zm = ZM_GMU.loc[ZM_GMU.mza!=0,cols] # Ignore summary rows
zm.rename(columns={'prom_ocup':'tot_integ','graproes':'nivelaprob','pocupada':'ocupados'}, inplace=True) # rename columns

# fill nans in gender
zm['pobmas'].fillna(zm['pobtot']-zm['pobfem'], inplace=True)
zm['pobfem'].fillna(zm['pobtot']-zm['pobmas'], inplace=True)

# fill nans using given population
zm['tothog'].fillna(round(zm['pobtot']/zm['tot_integ']), inplace=True)
zm['tot_integ'].fillna(zm['pobtot']/zm['tothog'], inplace=True)

# Turn false 0s to nans
a = (zm['tot_integ']==0) & (zm['pobtot']!=0)

zm.loc[a,'tothog'] = np.nan
zm.loc[a,'tot_integ'] = np.nan

# fill nans using means
colgroup = ['CVE_ZM','CVE_ENT','CVE_MUN','CVE_LOC','ageb']

zm['tothog'].fillna(-(-zm['pobtot']//zm.groupby(colgroup)['tot_integ'].transform(lambda x: x.fillna(x.mean()))), inplace=True)
zm['tothog'].fillna(-(-zm['pobtot']//zm.groupby('CVE_MUN')['tot_integ'].transform(lambda x: x.fillna(x.mean()))), inplace=True)
zm['tot_integ'].fillna(zm['pobtot']/zm['tothog'], inplace=True)

# Create new fields
zm['h_m_sexo'] = (zm['pobmas'] - zm['pobfem'])/zm['pobtot']
zm['ma_me_edad'] = (zm['p_18ymas']*2 - zm['pobtot'])/zm['pobtot']
zm['ocupados'] = zm['ocupados']/zm['pobtot']

zm['edad'] = (zm['p_0a2'].fillna(0)*1 + zm['p_3a5'].fillna(0)*4 + zm['p_6a11'].fillna(0)*8.5 + zm['p_12a14'].fillna(0)*13 +\
            zm['p_15a17'].fillna(0)*16 + zm['p_18a24'].fillna(0)*21 + zm['p_25a59'].fillna(0)*38 +\
            (zm['p_60ymas'].fillna(0)-zm['pob65_mas'].fillna(0))*62.5 + zm['pob65_mas'].fillna(0)*76 +\
            (zm['p_18ymas'].fillna(0)-(zm['p_18a24'].fillna(0) + zm['p_25a59'].fillna(0)+zm['p_60ymas'].fillna(0)))*41)/\
            (zm['p_0a2'].fillna(0) + zm['p_3a5'].fillna(0) + zm['p_6a11'].fillna(0)+zm['p_12a14'].fillna(0) + zm['p_15a17'].fillna(0) +\
            zm['p_18a24'].fillna(0) + zm['p_25a59'].fillna(0) + zm['p_60ymas'].fillna(0) +\
            (zm['p_18ymas'].fillna(0)-(zm['p_18a24'].fillna(0)+zm['p_25a59'].fillna(0)+zm['p_60ymas'].fillna(0))))

zm.drop(ZM_GMU.columns[18:31].tolist() , axis=1, inplace=True)

# set 0s in 0 pobtot
zm.loc[zm['pobtot']==0,['nivelaprob','ocupados','h_m_sexo','ma_me_edad','edad']] = 0
zm.loc[(zm.edad==0) & (zm.pobtot!=0), ['nivelaprob','ocupados','ma_me_edad','edad']] = np.nan

# fill nans using means
colnans = ['IMU','nivelaprob','ocupados','h_m_sexo','ma_me_edad','edad']
colgroup = ['CVE_ZM','CVE_ENT','CVE_MUN','CVE_LOC','ageb']
zm[colnans] = zm.groupby(colgroup)[colnans].transform(lambda x: x.fillna(x.mean()))
zm[colnans] = zm.groupby('CVE_MUN')[colnans].transform(lambda x: x.fillna(x.mean()))

zm.loc[(zm.edad==0) & (zm.pobtot!=0), ['nivelaprob','ocupados','ma_me_edad','edad']] = np.nan

# fill nans using means
colnans = ['IMU','nivelaprob','ocupados','h_m_sexo','ma_me_edad','edad']
colgroup = ['CVE_ZM','CVE_ENT','CVE_MUN','CVE_LOC','ageb']
zm[colnans] = zm.groupby('CVE_MUN')[colnans].transform(lambda x: x.fillna(x.mean()))

zm.loc[zm['pobtot']==0,['nivelaprob','ocupados','h_m_sexo','ma_me_edad','edad']] = np.nan

zm.to_pickle(pkls + 'ZM_REG1.pkl')

## Encuesta Nacional de Ingresos y Gastos de los Hogares 2016 
The ENIGH gives statistical data related to expenses and income. It also includes data related to the number of vehicles, electronic and electric devices, and an updated estimate of the population.

This data is not added yet to de previous dataframe as it is available at a lower level and it has to be first processed.

These files were downloaded from:

https://www.inegi.org.mx/programas/enigh/nc/2016/default.html#Microdatos

### Concentrador hogar

In [26]:
# Columns to take from Concentradorhogar
concentrador_cols = [0,1,2,3,5] + list(range(7,24)) + [57]

# Read concentrador file
concentrador_file = 'Ingresos y Gastos de los Hogares/2016/concentradohogar.csv'
concentrador = pd.read_csv(concentrador_file, usecols=concentrador_cols, na_values=' ')

# Change ageb format
concentrador['ageb'] = concentrador['ageb'].str[:3] + concentrador['ageb'].str[-1]

# Change female to 0 (male 1)
concentrador.loc[concentrador['sexo_jefe']==2,'sexo_jefe'] = 0

### Población
This section calculates values for age and education

In [27]:
# Columns to take from poblacion
poblacion_cols = [0,1,2,5,40]

# Read poblacion file
poblacion_file = 'Ingresos y Gastos de los Hogares/2016/poblacion.csv'
poblacion = pd.read_csv(poblacion_file, usecols=poblacion_cols, na_values=' ')

# Get mean age
pop_edad = poblacion.groupby(['folioviv','foliohog'], sort=False, as_index=False)['edad'].mean()

# Transform to escolaridad
dic_edu = {0:0,1:0,2:6,3:9,4:12,5:15,6:15,7:16,8:18,9:21}
poblacion['nivelaprob'] = poblacion['nivelaprob'].map(dic_edu)
# Get mean education of pop > 15 y
pop_edu = poblacion.loc[poblacion['edad']>=15].groupby(['folioviv','foliohog'], sort=False, as_index=False)['nivelaprob'].mean()
# Get mean education whole pop to fill houses with no >15
pop_edu_men15 = poblacion.groupby(['folioviv','foliohog'], sort=False, as_index=False)['nivelaprob'].mean()

# merge
pop = pop_edad.merge(pop_edu, on=['folioviv','foliohog'], how='left')
pop = pop.merge(pop_edu_men15, on=['folioviv','foliohog'], how='left')

pop['nivelaprob_x'] = pop['nivelaprob_x'].fillna(pop['nivelaprob_y'])
pop.drop('nivelaprob_y', axis=1, inplace=True)
pop.rename(columns={'nivelaprob_x':'nivelaprob'}, inplace=True)

concentrador = concentrador.merge(pop, on=['folioviv','foliohog'], how='left')

### Viviendas

In [28]:
# Columns to take from Viviendas
viviendas_cols = [0,5,21,22,23,24,27,28,29] +list(range(46,53))

# Read viviendas file
viviendas_file = 'Ingresos y Gastos de los Hogares/2016/viviendas.csv'
vivienda = pd.read_csv(viviendas_file, usecols=viviendas_cols, na_values=' ')

# Change false from 2 to 0
vivienda.loc[vivienda['calent_sol']==2,'calent_sol'] = 0
vivienda.loc[vivienda['calent_gas']==2,'calent_gas'] = 0
vivienda.loc[vivienda['medidor_luz']==2,'medidor_luz'] = 0
vivienda.loc[vivienda['bomba_agua']==2,'bomba_agua'] = 0
vivienda.loc[vivienda['tanque_gas']==2,'tanque_gas'] = 0
vivienda.loc[vivienda['aire_acond']==2,'aire_acond'] = 0
vivienda.loc[vivienda['calefacc']==2,'calefacc'] = 0

# merge renta and estim_pago
vivienda['renta'] = vivienda['renta'].fillna(vivienda['estim_pago'])
del vivienda['estim_pago']

### Hogares

In [29]:
# Columns to take from Hogares
hogares_cols = [0,1] + list(range(39,47)) + list(range(57,97))

# Read Hogares file
hogares_file = 'Ingresos y Gastos de los Hogares/2016/hogares.csv'
hogares = pd.read_csv(hogares_file, usecols=hogares_cols, na_values=[' ','&',-1])

# Merge all kind of automobile
hogares['num_vehiculos'] = hogares['num_auto'] + hogares['num_van'] + hogares['num_pickup']
hogares['anio_vehiculos'] = hogares[['anio_auto', 'anio_van', 'anio_pickup']].mean(axis = 1, skipna = True).round()

hogares.drop(['num_auto','anio_auto','num_van','anio_van','num_pickup','anio_pickup'], axis=1, inplace=True)

### Gastos
Combustibles vehiculos
F007,F008,F009

Combustibles
G009-G014

Electricidad
R001

Gas
R003


In [30]:
# Columns to take from gastos
gastos_cols = [0,1,2,23]

# Expenses' keys
claves = ['R001','R003','G009','G010','G011','G012','G013','G014']
dic_cves = {'R001':'ele', 'R003':'gas', 'G009':'glp', 'G010':'pet', 'G011':'die', 'G012':'car', 'G013':'len', 'G014':'heat'}

# Read gastos file
gastos_file = 'Ingresos y Gastos de los Hogares/2016/gastoshogar.csv'
gastos = pd.read_csv(gastos_file, usecols=gastos_cols, na_values=' ')

gastos.dropna(subset=['gasto_tri'], inplace=True)

# Convert every type of expense into a column
gastolist = []
for clave in claves:
    gasto_x = gastos.loc[gastos['clave']==clave].copy()
    gasto_x.rename(columns={'gasto_tri':'gasto_tri_' + dic_cves[clave]}, inplace=True)
    gasto_x.drop(['clave'], axis=1, inplace=True)
    gastolist.append(gasto_x)
del gastos

# merge all the expenses
gastos = reduce(lambda  left,right: pd.merge(left,right, on=['folioviv','foliohog'], how='outer'), gastolist)

### ENIGH 2016

In [31]:
# merge all tables
gast_ing = pd.merge(pd.merge(pd.merge(concentrador,vivienda, on='folioviv'), 
                             hogares, on=['folioviv','foliohog']), gastos, on=['folioviv','foliohog'])

del concentrador, vivienda, hogares, gastos

# Explode ubica_geop into columns to create matching index to ZM_2015
gast_ing['ubica_geo'] = gast_ing['ubica_geo'].astype(str) 

gast_ing['CVE_ENT'] = gast_ing['ubica_geo'].str[:-7].astype(np.int64)
gast_ing['CVE_MUN'] = gast_ing['ubica_geo'].str[:-4].astype(np.int64)
gast_ing['CVE_LOC'] = gast_ing['ubica_geo'].str[-4:].astype(np.int64)
del gast_ing['ubica_geo']

# Select areas within Metropolitan zones
gast_ing_ZM = gast_ing.loc[gast_ing['CVE_MUN'].isin(ZM_2015['CVE_MUN'])]

gast_ing_ZM.to_pickle(pkls + 'gast_ing_ZM.pkl')

## DENUEs
The Directorio Estadístico Nacional de Unidades Económicas (DENUE) is a directory of the active econimic units within the country, including idetifiers, location, economic activity, size, etc. The information of 2015 was taken as it is the closest registered to the CPV2010.

These files were downloaded from:
https://www.inegi.org.mx/app/descarga/?ti=6

In [32]:
# Columns to take from DENUEs
denues_cols = [0,1,3,4,5,22,26,28,30,32,33,35] 

# Get csv files in directory
denues_path = 'DENUE/2015'
denues_files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(denues_path):
    for file in f:
        if '.csv' in file:
            denues_files.append(os.path.join(r, file))
            
# Put all csv files in a list to concatenate it
denues_list = []
for file in denues_files:
    denues_x = pd.read_csv(file, usecols=denues_cols, dtype={'Área geoestadística básica ': str}, na_values=[' '])
    denues_list.append(denues_x)

denues = pd.concat(denues_list)

# change column names
denues.rename(columns={'Nombre de la Unidad Económica':'nom_ue', 'Código de la clase de actividad SCIAN':'cod_scian',
                      'Nombre de clase de la actividad':'actividad', 'Descripcion estrato personal ocupado':'num_personal',
                      'Tipo centro comercial':'centro_com', 'Área geoestadística básica ':'ageb'}, inplace=True)

# change municipio code
denues['Clave municipio'] = denues['Clave municipio'] + denues['Clave entidad']*1000

# Create dataframe with DENUEs within the metropolitan zones
denues_ZM = denues.loc[denues['Clave municipio'].isin(ZM_2015['CVE_MUN'])]

denues_ZM.to_pickle(pkls + 'denues_ZM.pkl')

denues_ZM.head()

## Load pkls

In [6]:
denues_ZM = pd.read_pickle(pkls + 'denues_ZM.pkl')
gast_ing_ZM = pd.read_pickle(pkls + 'gast_ing_ZM.pkl')
ZM_2015 = pd.read_pickle(pkls + 'ZM_2015.pkl')
ZM_GMU = pd.read_pickle(pkls + 'ZM_GMU.pkl')

# Shapefiles
This section merge the shapefiles of the Marco Geoestadistico into sigle shapefiles, not necessary if the shapefiles already exist

## Municipalities

In [122]:
# Get csv files in directory
mun_geo_path = 'Marco Geoestadistico/conjunto de datos'
mun_files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(mun_geo_path):
    for file in f:
        if 'mun.shp' in file:
            mun_files.append(os.path.join(r, file))

# Put all csv files in a list to concatenate it            
mun_list = []
for file in mun_files:
    mun_x = gpd.read_file(file)
    mun_list.append(mun_x)

mun = pd.concat(mun_list)

mun.reset_index(inplace=True, drop=True)

# Fix CVE_MUN columns
mun['CVE_MUN'] = mun['CVEGEO'].astype(np.int64)
del mun['CVEGEO']

mun.to_file("shapefiles/MUN/MUN.shp")

## Metropolitan areas: Municipalities

In [124]:
# Take metropolitan municipalities
mun_ZM = mun.loc[mun['CVE_MUN'].isin(ZM_2015['CVE_MUN'])].copy()

# Merge Geodataframe with metropolitan zones dataframe
mun_ZM = mun_ZM.merge(ZM_2015[['CVE_MUN','CVE_ZM','NOM_ZM']], on='CVE_MUN')

# Dissolve into metropolitan zones
ZM = mun_ZM.dissolve(by='CVE_ZM', aggfunc='first')
ZM.drop(['CVE_ENT', 'CVE_MUN', 'NOMGEO'], axis=1, inplace=True)

ZM.reset_index(inplace=True)

ZM.to_file("shapefiles/ZM/ZM.shp")

## Metropolitan areas: Blocks

In [10]:
# Get csv files in directory
mzn_geo_path = 'Marco Geoestadistico/conjunto de datos'
mzn_files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(mzn_geo_path):
    for file in f:
        if 'm.shp' in file and len(file)==7:
            mzn_files.append(os.path.join(r, file))

# Put all csv files in a list to concatenate it
mnz_list = []
for file in mzn_files:
    mnz_x = gpd.read_file(file)
    mnz_list.append(mnz_x)

mnz = pd.concat(mnz_list)
mnz.reset_index(inplace=True, drop=True)

mnz['CVE_MUN'] = (mnz['CVE_ENT'] + mnz['CVE_MUN']).astype(np.int64)
mnz['CVE_ENT'] = mnz['CVE_ENT'].astype(np.int64)
mnz['CVE_LOC'] = mnz['CVE_LOC'].astype(np.int64)

# Take metropolitan municipalities
mzn_ZM = mnz.loc[mnz['CVE_MUN'].isin(ZM_2015['CVE_MUN'])].copy()

# Merge Geodataframe with metropolitan zones dataframe
mzn_ZM = mzn_ZM.merge(ZM_2015[['CVE_MUN','CVE_ZM','NOM_ZM']], on='CVE_MUN')

mzn_ZM.head()

In [17]:
mzn_ZM.to_file("shapefiles/manzanas/mzn_ZM.shp")
mzn_ZM.to_pickle(pkls + 'mza_shp.pkl')